## **Import Required Libraries**

In [ ]:
# Import pandas
import pandas as pd

# Import libraries for web scraping
import requests
from bs4 import BeautifulSoup

# Import wikipedia
import wikipedia

# Import matplotlib for plotting
import matplotlib.pyplot as plt

# Import json to load config file
import json

## **Creates French Election Data**

### **Import Raw Data**

In [ ]:
# Load the file path from the config.json file
with open('config.json', 'r') as file:
    config = json.load(file)

# Access the input directory path
inp_dir = config['input_data_dir']

# Import recent election results raw data
fr_re_raw = pd.read_csv(f"{inp_dir}France_Election_Results.csv")

### **Clean Data**

In [ ]:
# Create copy of data to modify
fr_re = fr_re_raw.copy()

# Cast Votes cols as float
fr_re["First Round Votes"] = fr_re["First Round Votes"].str.replace(",","").astype(float)
fr_re["Second Round Votes"] = fr_re["Second Round Votes"].str.replace(",","").astype(float)

### **Create Function to Automatically Retrieve Alignments**

In [ ]:
# Define function to get alignment of a political party
def pol_alignment(party):
    # Get url for party's wiki page
    try:
        url = wikipedia.page(party).url
    except:
        return None

    # Fetch the Wikipedia page
    response = requests.get(url)

    # Parse the HTML
    soup = BeautifulSoup(response.content, "html.parser")

    # Define the infobox
    infobox = soup.find('table', {'class': 'infobox'})

    if infobox is None:
        return None
    else:
        # Initialise political position
        position = None

        # Find the political position from the infobox
        for row in infobox.find_all('tr'):
            if "Political\xa0position" in row.get_text():
                position = row.get_text().split("Political\xa0position")[1]
                position = position.split("[")[0]

    return position

### **Apply Function**

In [ ]:
# Create mapping using function
party_alignment = {party: pol_alignment(party) for party in fr_re["Party"]}

# Get keys with None values
none_keys = [key for key, value in party_alignment.items() if value is None]
print(none_keys)

### **Manually Update Unmapped Parties**

In [ ]:
# Create mapping for unmapped parties
unmapped_parties = {
    "National Rally": "Far-right",
    "Union of the Far-Right": "Far-right",
    "The Republicans": "Centre-right to right-wing",
    "Miscellaneous right": "Right-wing",
    "Miscellaneous left": "Left-wing",
    "Miscellaneous centre": "Centre",
    "Miscellaneous far-left": "Far-left",
    "Regionalists": None,
    "Ecologists": None,
    "Miscellaneous": None,
    "Sovereignist right": "Right-wing",
    "Miscellaneous far-right": "Far-right"
}

# Combine the mappings
party_alignment = party_alignment | unmapped_parties

### **Add Alignment of Each Party**

In [ ]:
# Create numerical mapping of Alignment
#   ranges from -6 (Far-Left) to 6 (Far-Right)
alignment_mapping = {
    "Centre-left": -2,
    "Centre-right": 2,
    "Centre": 0,
    "Right-wing": 4,
    "Left-wing": -4,
    "Centre to centre-left": -1,
    "Centre to centre-right": 1,
    "Centre-right to right-wing": 3,
    "Center-left": -2,
    "Big tent": 0,
    "Centre-left to left-wing": -3,
    "Far-left": -6,
    "Centre-rightFactions:Right-wing": 3,
    "CentreFactions:Left-wing and right-wing": 0,
    "\nMajority:\nRight-wing": 4,
    "Right-wing to far-right": 5,
    "Center-right": 2,
    "Center-right to right-wing": 3,
    "Far-right": 6,
    "Left-Wing": -4,
    "Far-Right": 6,
    "Syncretic": 0,
    "Single-issue": "No Data",
    None: "No Data",        # PLACEHOLDER: CHANGE!
}

# Map Party to alignment
fr_re["Alignment"] = fr_re["Party"].map(party_alignment)

# Map alignment to numerical alignment
fr_re["Alignment Score"] = fr_re["Alignment"].map(alignment_mapping)

# Check that all valid parties have been mapped
print(f"Number of unmapped alignments: {len(fr_re[(~fr_re['Alignment'].isna()) & (fr_re['Alignment Score'].isna())])}")

# Preview data
fr_re.head()

### **Create Spectrum Results Data**

In [ ]:
#Get total votes over both rounds
fr_re["Total Votes"] = fr_re["First Round Votes"] + fr_re["Second Round Votes"]

# Get vote count by alignment
align_votes = fr_re.groupby(['Alignment Score'])['Total Votes'].sum().sort_values(ascending=False).reset_index()

# Remove "No Data" Alignment
align_votes = align_votes[align_votes["Alignment Score"] != 'No Data']

# Create % of total column
align_votes['In-Scope Vote Prop'] = (align_votes['Total Votes'] / align_votes['Total Votes'].sum())
align_votes['In-Scope Vote %'] = (align_votes['Total Votes'] / align_votes['Total Votes'].sum()) * 100
align_votes['In-Scope Vote %'] = align_votes['In-Scope Vote %'].round(2)

# View data
align_votes

## **Data Visualisation**

### **Give Alignment Score**

In [ ]:
# Give alignment score for French 2024 election
align_score = align_votes["Alignment Score"].dot(align_votes["In-Scope Vote Prop"])
print(f"Alignment score of the French 2024 Election: {align_score}")

## **Export Data**

In [ ]:
# Access the input directory path
out_dir = config['inter_outputs_dir']

align_votes.to_csv(f"{out_dir}France_New.csv", index=False)